# KMeans

In [ ]:
import numpy as np
import pandas as pd
import time

### Create synthetic dataset

In [ ]:
n_rows = 1000000
n_cols = 500
n_clusters_data = 200
cluster_std = 1.0
dtype='float32'
from sklearn.datasets import make_blobs
data, _ = make_blobs(
        n_samples=n_rows, n_features=n_cols, centers=n_clusters_data, cluster_std=cluster_std, random_state=0
    )  # make_blobs creates a random dataset of isotropic gaussian blobs.

data = data.astype(np.float32)

### Convert dataset to Spark DataFrame

In [ ]:
pd_data = pd.DataFrame({"features": list(data)})
df = spark.createDataFrame(pd_data)

### We will use this function to build both the Spark RAPIDS ML (GPU) and Spark ML (CPU) linear estimator objects, demonstrating the common API

In [ ]:
def build_kmeans_estimator(estimator_class):
    return ( 
            estimator_class()
            .setTol(1.0e-20)
            .setK(200)
            .setFeaturesCol("features")
            .setMaxIter(15)
           )

## Spark RAPIDS ML (GPU)

In [ ]:
from spark_rapids_ml.clustering import KMeans
gpu_kmeans = build_kmeans_estimator(KMeans)

Estimator can be persisted and reloaded.

In [ ]:
estimator_path = "/tmp/kmeans-estimator"

In [ ]:
gpu_kmeans.write().overwrite().save(estimator_path)
gpu_kmeans_loaded = KMeans.load(estimator_path)

### Fit

In [ ]:
start_time = time.time()
gpu_model = gpu_kmeans_loaded.fit(df)
print(f"Fit took: {time.time() - start_time} sec")

In [ ]:
gpu_kmeans_loaded.getK()

In [ ]:
sorted_clusters = sorted([vec.tolist() for vec in gpu_model.clusterCenters()])

In [ ]:
[vec[0:10] for vec in sorted_clusters[0:2]]

### Transform

In [ ]:
model_path = "/tmp/kmeans-model"

In [ ]:
gpu_model.write().overwrite().save(model_path)

In [ ]:
gpu_model_loaded = gpu_model.read().load(model_path)

In [ ]:
[vec[0:10] for vec in sorted(gpu_model_loaded.cluster_centers_)[0:2]]

In [ ]:
transformed_df = gpu_model_loaded.setPredictionCol("transformed").transform(df)

In [ ]:
transformed_df.printSchema()

In [ ]:
transformed_df.count()

In [ ]:
transformed_df.show(10)

## Spark ML (CPU)

In [ ]:
from pyspark.ml.clustering import KMeans
cpu_kmeans = build_kmeans_estimator(KMeans)

Convert array sql type to VectorUDT Dataframe expected by Spark ML algos (Note: Spark RAPIDS ML also accepts VectorUDT Dataframes in addition to array type Dataframe above, along with a scalar column format - see docs).

In [ ]:
from pyspark.ml.functions import array_to_vector

In [ ]:
vector_df = df.select(array_to_vector(df.features).alias("features"))

### Fit

In [ ]:
start_time = time.time()
cpu_kmeans_model = cpu_kmeans.fit(vector_df)
print(f"Fit took: {time.time() - start_time} sec")

In [ ]:
type(cpu_kmeans_model.clusterCenters()[0])

In [ ]:
sorted_cpu_cluster_centers = sorted([vec.tolist() for vec in cpu_kmeans_model.clusterCenters()])
[vec[0:10] for vec in sorted_cpu_cluster_centers[0:2]]

### Transform

In [ ]:
spark_transformed = cpu_kmeans_model.setPredictionCol("transformed").transform(vector_df)

In [ ]:
spark_transformed.filter(spark_transformed.transformed >= 0).count()

In [ ]:
spark_transformed.show(10)